In [1]:
import time, random
import numpy as np
# import pennylane as qml
# from qiskit import Aer, transpile, execute
from qiskit.circuit import QuantumCircuit
from qiskit.quantum_info import random_clifford, Pauli, Statevector
import matplotlib.pyplot as plt

np.set_printoptions(precision=6, edgeitems=10, linewidth=150, suppress=True)

In [2]:
import qiskit
import itertools
from qiskit import *
from qiskit.quantum_info import Clifford, random_clifford
from qiskit.synthesis import synth_clifford_full
from qiskit.quantum_info import hellinger_fidelity as hf

from utils.pauli_checks import ChecksFinder, add_pauli_checks, add_meas_pauli_checks, add_linear_meas_pauli_checks,  search_for_pauli_list
from utils.pauli_checks import gen_initial_layout, gen_final_layout, filter_results, pauli_strings_commute

from utils.utils import norm_dict, total_counts
# from utils.vqe_utils import evaluation
from utils.postprocess import singlecheck_postprocess, rightchecks_postprocess, filter_results_reindex

#### I. Calibrating $\tilde{f}$ in the noisy Clifford channel using hardware

In [3]:
total_trials = 10000
num_qubits = 4
def calibration_circuit(Clifford):
    qc = QuantumCircuit(num_qubits)
    
    clifford_circuit = Clifford.to_circuit()
    qc.compose(clifford_circuit, qubits=[0,1,2,3], inplace=True)
    
    qc.measure_all()
    return qc

In [4]:
cali_C_list = []
for i in range(total_trials):
    Clifford = random_clifford(4)
    cali_C_list.append(Clifford)
    
cali_circs = []
for i in range(total_trials):
    circuit = calibration_circuit(cali_C_list[i])
    cali_circs.append(circuit)

In [5]:
# from qiskit_ibm_runtime import Session, Options, SamplerV2 as Sampler
from qiskit_ibm_runtime import Session, Sampler, Options
from qiskit_ibm_runtime.fake_provider import *
from qiskit_aer import AerSimulator
import qiskit_aer.noise as noise
from itertools import combinations

# Make a noise model
fake_backend = FakeCairo() # select backend
noise_model = noise.NoiseModel.from_backend(fake_backend)

options = Options(optimization_level=2, resilience_level=1) # choose the proper levels on hardware
options.simulator = {
    "noise_model": noise_model,
    "basis_gates": fake_backend.configuration().basis_gates,
    "coupling_map": fake_backend.configuration().coupling_map,
    "seed_simulator": 42
}

# backend = service.get_backend("") 
# backend = "ibmq_qasm_simulator" # use the simulator for now
backend = AerSimulator()

In [6]:
with Session(backend=backend) as session:
    sampler = Sampler(session=session, options=options)

    # define physical qubits to be used in the layout arguement
    job = sampler.run(cali_circs, shots=100)
    print(f"Job ID: {job.job_id()}")
    print(f">>> Job Status: {job.status()}")

    result = job.result()

    # Close the session only if all jobs are finished
    # and you don't need to run more in the session.
    session.close()

/Users/celsloaner/opt/anaconda3/envs/pcs_zne/lib/python3.9/site-packages/qiskit_ibm_runtime/session.py:157: UserWarning: Session is not supported in local testing mode or when using a simulator.
  warnings.warn(
/var/folders/b4/nbfyyzvd3tlcp6vv5xyc9wpw0000gq/T/ipykernel_26514/3881146939.py:2: DeprecationWarning: The Sampler and Estimator V1 primitives have been deprecated as of qiskit-ibm-runtime 0.23.0 and will be removed no sooner than 3 months after the release date. Please use the V2 Primitives. See the `V2 migration guide <https://docs.quantum.ibm.com/api/migration-guides/v2-primitives>`_. for more details
  sampler = Sampler(session=session, options=options)


Job ID: 95283150-0b32-4da2-8b04-d6b56be7ce24
>>> Job Status: JobStatus.RUNNING


In [7]:
cali_b_lists = []

for i in range(total_trials):
    di = {}
    for key in list(result.quasi_dists[i].binary_probabilities().keys()):
        di.update({key[:num_qubits]: result.quasi_dists[i].binary_probabilities().get(key)})
    cali_b_lists.append(di)

In [8]:
def calibrating_f(cali_b_lists, cali_C_list, num_qubits):
    d = 2**num_qubits
    num_snapshots = len(cali_C_list)
    
    f_tilde = 0.
    for b_dict, clifford in zip(cali_b_lists, cali_C_list):
        F = computing_F(b_dict, clifford, num_qubits)
        f_tilde += np.real((d*F - 1) / (d - 1))
    
    return f_tilde / num_snapshots


def computing_F(b_dict, clifford, num_qubits):
    zero_state = state_reconstruction('0'*num_qubits)
    U = clifford.to_matrix()
    
    F = 0. + 0.j
    denom = 0.
    for b_state in list(b_dict.keys()):
        F += np.trace(zero_state @ U.T.conj() @ state_reconstruction(b_state) @ U) * b_dict.get(b_state)
        denom += b_dict.get(b_state)
    return F / denom


def state_reconstruction(b_str: str):
    '''
    '''
    zero_state = np.array([[1,0],[0,0]])
    one_state = np.array([[0,0], [0,1]])
    rho = [1]
    for i in b_str:
        state_i = zero_state if i=='0' else one_state
        rho = np.kron(rho, state_i)
    return rho

In [9]:
%%time

f_tilde = calibrating_f(cali_b_lists, cali_C_list, num_qubits)
print(f'The calibrated f_tilde is {f_tilde}; while the noiseless reference is {1/(2**num_qubits+1)}')

The calibrated f_tilde is 0.04866800000000035; while the noiseless reference is 0.058823529411764705
CPU times: user 3min 16s, sys: 562 ms, total: 3min 17s
Wall time: 3min 19s


#### II. Perform the standard shadow experiments

In [10]:
# define the ansatz circuit

def hartree_fock_circuit(num_qubits):
    qc = QuantumCircuit(num_qubits)
    # prepare the Hartree-Fock state
    qc.x(0)
    qc.x(1)
    return qc

#the circuit without hartree fock preperation
def hydrogen_trial_circuit_noprep(num_qubits):
    qc = QuantumCircuit(num_qubits)
#     # prepare the Hartree-Fock state
#     qc.x(0)
#     qc.x(1)
    
    qc.rx(np.pi/2, 0)
    qc.h(1)
    qc.h(2)
    qc.h(3)
    
    qc.cx(0,1)
    qc.cx(1,2)
    qc.cx(2,3)
    
    qc.rz(1.0, 3)
    
    qc.cx(2,3)
    qc.cx(1,2)
    qc.cx(0,1)
    
    qc.rx(-np.pi/2, 0)
    qc.h(1)
    qc.h(2)
    qc.h(3)
    
    return qc

def hydrogen_trial_circuit(num_qubits):
    qc = QuantumCircuit(num_qubits)
    # prepare the Hartree-Fock state
    qc.x(0)
    qc.x(1)
    
    qc.rx(np.pi/2, 0)
    qc.h(1)
    qc.h(2)
    qc.h(3)
    
    qc.cx(0,1)
    qc.cx(1,2)
    qc.cx(2,3)
    
    qc.rz(1.0, 3)
    
    qc.cx(2,3)
    qc.cx(1,2)
    qc.cx(0,1)
    
    qc.rx(-np.pi/2, 0)
    qc.h(1)
    qc.h(2)
    qc.h(3)
    
    return qc


def hydrogen_shadow_circuit(Clifford, num_qubits):
    qc = hydrogen_trial_circuit(num_qubits)
    
    clifford_circuit = Clifford.to_circuit()
    qc.compose(clifford_circuit, qubits=[0,1,2,3], inplace=True)
    
    qc.measure_all()
    return qc

def hydrogen_shadow_PCS_circuit(Clifford, num_qubits, num_checks, single_side = False):
    total_qubits = num_qubits + num_checks
    
    qc = hydrogen_trial_circuit(total_qubits)

    clif_qc = Clifford.to_circuit()
    
    characters = ['I', 'Z']
    strings = [''.join(p) for p in itertools.product(characters, repeat=num_qubits)]
    
    test_finder = ChecksFinder(num_qubits, clif_qc)
    p1_list = []
    for string in strings:
        string_list = list(string)
        result = test_finder.find_checks_sym(pauli_group_elem = string_list)
        #print(result.p1_str, result.p2_str)
        p1_list.append([result.p1_str, result.p2_str])
        
    sorted_list = sorted(p1_list, key=lambda s: s[1].count('I'))
    pauli_list = sorted_list[-num_qubits -1:-1]
    
    #
    initial_layout = {}
    for i in range(0, num_qubits):
        initial_layout[i] = [i]

    final_layout = {}
    for i in range(0, num_qubits):
        final_layout[i] = [i]
        
    #add pauli check on two sides:
    #specify the left and right pauli strings
    pcs_qc_list = []
    sign_list = []
    pl_list = []
    pr_list = []

    for i in range(0, num_checks):
        pl = pauli_list[i][0][2:]
        pr = pauli_list[i][1][2:]
        if i == 0:
            temp_qc = add_pauli_checks(clif_qc, pl, pr, initial_layout, final_layout, False, single_side, False, False, False, 0)
            save_qc = add_pauli_checks(clif_qc, pl, pr, initial_layout, final_layout, False, single_side, False, False, False, 0)
            prev_qc = temp_qc
        else:
            temp_qc = add_pauli_checks(prev_qc, pl, pr, initial_layout, final_layout, False, single_side, False, False, False, 0)
            save_qc = add_pauli_checks(prev_qc, pl, pr, initial_layout, final_layout, False, single_side, False, False, False, 0) 
            prev_qc = temp_qc
        pl_list.append(pl)
        pr_list.append(pr)
        sign_list.append(pauli_list[i][0][:2])
        pcs_qc_list.append(save_qc)

    
    qc.compose(pcs_qc_list[-1], qubits=[i for i in range(0, total_qubits)], inplace=True)
    
    qc.measure_all()
    return qc, (sign_list, pl_list, pr_list)

In [11]:
def hydrogen_shadow_PCS_checkprep(Clifford, num_qubits, num_checks, single_side=False):
    total_qubits = num_qubits + num_checks
    qc_prep = hartree_fock_circuit(total_qubits)
    qc = hydrogen_trial_circuit_noprep(total_qubits)
    clif_qc = Clifford.to_circuit()
    
    # Generate all combinations of 'I' and 'Z' for num_qubits
    characters = ['I', 'Z']
    strings = [''.join(p) for p in itertools.product(characters, repeat=num_qubits)]
    
    test_finder = ChecksFinder(num_qubits, clif_qc)
    p1_list = []
    for string in strings:
        string_list = list(string)
        result = test_finder.find_checks_sym(pauli_group_elem = string_list)
        #print(result.p1_str, result.p2_str)
        p1_list.append([result.p1_str, result.p2_str])
        
    sorted_list = sorted(p1_list, key=lambda s: s[1].count('I'))
    pauli_list = sorted_list[-num_qubits -1:-1]
    
    initial_layout = {i: [i] for i in range(num_qubits)}
    final_layout = initial_layout.copy()

    #add pauli check on two sides:
    #specify the left and right pauli strings
    pcs_qc_list = []
    sign_list = []
    pl_list = []
    pr_list = []
            
    commute_pls, commute_prs, commute_signs, anticommute_pls, anticommute_prs, anticommute_signs = classify_pauli_checks(pauli_list, num_checks, prep_str = 'XXXY')

            
    # first add the anticommute checks in the middle of the circuit
    for j in range(0, len(anticommute_pls)):
        pl = anticommute_pls[j]
        pr = anticommute_prs[j]
        sign = anticommute_signs[j]
        if j == 0:
            temp_qc = add_pauli_checks(clif_qc, pl, pr, initial_layout, final_layout, False, single_side, False, False, False, 0)
            save_qc = add_pauli_checks(clif_qc, pl, pr, initial_layout, final_layout, False, single_side, False, False, False, 0)
            prev_qc = temp_qc
        else:
            temp_qc = add_pauli_checks(prev_qc, pl, pr, initial_layout, final_layout, False, single_side, False, False, False, 0)
            save_qc = add_pauli_checks(prev_qc, pl, pr, initial_layout, final_layout, False, single_side, False, False, False, 0) 
            prev_qc = temp_qc
        pl_list.append(pl)
        pr_list.append(pr)
        sign_list.append(sign)
        pcs_qc_list.append(save_qc)

    if len(anticommute_pls) > 0:
        qc.compose(pcs_qc_list[-1], qubits=[i for i in range(0, num_qubits + len(anticommute_pls))], inplace=True)
    
    # then add the commute checks at the beginning of the circuit
    num_commute = len(commute_pls)
    for k in range(0, len(commute_pls)):
        pl = commute_pls[k]
        pr = commute_prs[k]
        sign = commute_signs[k]
        if k == 0:
            temp_qc = add_pauli_checks(qc, pl, pr, initial_layout, final_layout, False, single_side, False, False, False, num_commute - k)
            save_qc = add_pauli_checks(qc, pl, pr, initial_layout, final_layout, False, single_side, False, False, False, num_commute - k)
            prev_qc = temp_qc
        else:
            temp_qc = add_pauli_checks(prev_qc, pl, pr, initial_layout, final_layout, False, single_side, False, False, False, num_commute - k)
            save_qc = add_pauli_checks(prev_qc, pl, pr, initial_layout, final_layout, False, single_side, False, False, False, num_commute - k) 
            prev_qc = temp_qc
        pl_list.append(pl)
        pr_list.append(pr)
        sign_list.append(sign)
        pcs_qc_list.append(save_qc)
    
    if len(commute_pls) > 0:
        qc = pcs_qc_list[-1]
        
    if len(anticommute_pls) == 0:
        qc.compose(clif_qc, inplace=True)

    qc_prep.compose(qc, inplace=True)
    qc_prep.measure_all()
    return qc_prep, (sign_list, pl_list, pr_list)

def classify_pauli_checks(pauli_list, num_checks, prep_str):
    """
    Classifies Pauli checks into commuting and anti-commuting groups based on the preparation string.
    
    Parameters:
    - pauli_list: List of Pauli checks
    - num_checks: Number of checks to classify
    - prep_str: Preparation string to determine commuting or anti-commuting
    
    Returns:
    - Tuple of lists: (commute_pls, commute_prs, commute_signs, anticommute_pls, anticommute_prs, anticommute_signs)
    """
    commute_pls, commute_prs, commute_signs = [], [], []
    anticommute_pls, anticommute_prs, anticommute_signs = [], [], []
    
    for i in range(num_checks):
        pl, pr = pauli_list[i][0][2:], pauli_list[i][1][2:]
        sign = pauli_list[i][0][:2]
        
        if pauli_strings_commute(pl, prep_str):
            commute_pls.append(pl)
            commute_prs.append(pr)
            commute_signs.append(sign)
        else:
            anticommute_pls.append(pl)
            anticommute_prs.append(pr)
            anticommute_signs.append(sign)
    
    return (commute_pls, commute_prs, commute_signs, anticommute_pls, anticommute_prs, anticommute_signs)

    commute_pls, commute_prs, commute_signs, anticommute_pls, anticommute_prs, anticommute_signs = classify_pauli_checks(pauli_list, num_checks, 'XXXY')
    

In [12]:
num_qubits = 4
num_checks = 4
C_list = []
for i in range(total_trials):
    Clifford = random_clifford(4)
    C_list.append(Clifford)
circs_list = []
checks_list = []
for check_id in range(1, num_checks + 1):
    circs = []
    checks = []
    for i in range(total_trials):
        print(check_id, i)
        circ, check = hydrogen_shadow_PCS_circuit(C_list[i], num_qubits, check_id, True)
        circs.append(circ)
        checks.append(check)
    circs_list.append(circs)
    checks_list.append(checks)
    
orign_circs = []
for i in range(total_trials):
    circuit = hydrogen_shadow_circuit(C_list[i], num_qubits)
    orign_circs.append(circuit)

1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
1 12
1 13
1 14
1 15
1 16
1 17
1 18
1 19
1 20
1 21
1 22
1 23
1 24
1 25
1 26
1 27
1 28
1 29
1 30
1 31
1 32
1 33
1 34
1 35
1 36
1 37
1 38
1 39
1 40
1 41
1 42
1 43
1 44
1 45
1 46
1 47
1 48
1 49
1 50
1 51
1 52
1 53
1 54
1 55
1 56
1 57
1 58
1 59
1 60
1 61
1 62
1 63
1 64
1 65
1 66
1 67
1 68
1 69
1 70
1 71
1 72
1 73
1 74
1 75
1 76
1 77
1 78
1 79
1 80
1 81
1 82
1 83
1 84
1 85
1 86
1 87
1 88
1 89
1 90
1 91
1 92
1 93
1 94
1 95
1 96
1 97
1 98
1 99
1 100
1 101
1 102
1 103
1 104
1 105
1 106
1 107
1 108
1 109
1 110
1 111
1 112
1 113
1 114
1 115
1 116
1 117
1 118
1 119
1 120
1 121
1 122
1 123
1 124
1 125
1 126
1 127
1 128
1 129
1 130
1 131
1 132
1 133
1 134
1 135
1 136
1 137
1 138
1 139
1 140
1 141
1 142
1 143
1 144
1 145
1 146
1 147
1 148
1 149
1 150
1 151
1 152
1 153
1 154
1 155
1 156
1 157
1 158
1 159
1 160
1 161
1 162
1 163
1 164
1 165
1 166
1 167
1 168
1 169
1 170
1 171
1 172
1 173
1 174
1 175
1 176
1 177
1 178
1 179
1 180
1 181
1 182
1 183
1 184


1 1330
1 1331
1 1332
1 1333
1 1334
1 1335
1 1336
1 1337
1 1338
1 1339
1 1340
1 1341
1 1342
1 1343
1 1344
1 1345
1 1346
1 1347
1 1348
1 1349
1 1350
1 1351
1 1352
1 1353
1 1354
1 1355
1 1356
1 1357
1 1358
1 1359
1 1360
1 1361
1 1362
1 1363
1 1364
1 1365
1 1366
1 1367
1 1368
1 1369
1 1370
1 1371
1 1372
1 1373
1 1374
1 1375
1 1376
1 1377
1 1378
1 1379
1 1380
1 1381
1 1382
1 1383
1 1384
1 1385
1 1386
1 1387
1 1388
1 1389
1 1390
1 1391
1 1392
1 1393
1 1394
1 1395
1 1396
1 1397
1 1398
1 1399
1 1400
1 1401
1 1402
1 1403
1 1404
1 1405
1 1406
1 1407
1 1408
1 1409
1 1410
1 1411
1 1412
1 1413
1 1414
1 1415
1 1416
1 1417
1 1418
1 1419
1 1420
1 1421
1 1422
1 1423
1 1424
1 1425
1 1426
1 1427
1 1428
1 1429
1 1430
1 1431
1 1432
1 1433
1 1434
1 1435
1 1436
1 1437
1 1438
1 1439
1 1440
1 1441
1 1442
1 1443
1 1444
1 1445
1 1446
1 1447
1 1448
1 1449
1 1450
1 1451
1 1452
1 1453
1 1454
1 1455
1 1456
1 1457
1 1458
1 1459
1 1460
1 1461
1 1462
1 1463
1 1464
1 1465
1 1466
1 1467
1 1468
1 1469
1 1470
1 1471
1 1472

1 2501
1 2502
1 2503
1 2504
1 2505
1 2506
1 2507
1 2508
1 2509
1 2510
1 2511
1 2512
1 2513
1 2514
1 2515
1 2516
1 2517
1 2518
1 2519
1 2520
1 2521
1 2522
1 2523
1 2524
1 2525
1 2526
1 2527
1 2528
1 2529
1 2530
1 2531
1 2532
1 2533
1 2534
1 2535
1 2536
1 2537
1 2538
1 2539
1 2540
1 2541
1 2542
1 2543
1 2544
1 2545
1 2546
1 2547
1 2548
1 2549
1 2550
1 2551
1 2552
1 2553
1 2554
1 2555
1 2556
1 2557
1 2558
1 2559
1 2560
1 2561
1 2562
1 2563
1 2564
1 2565
1 2566
1 2567
1 2568
1 2569
1 2570
1 2571
1 2572
1 2573
1 2574
1 2575
1 2576
1 2577
1 2578
1 2579
1 2580
1 2581
1 2582
1 2583
1 2584
1 2585
1 2586
1 2587
1 2588
1 2589
1 2590
1 2591
1 2592
1 2593
1 2594
1 2595
1 2596
1 2597
1 2598
1 2599
1 2600
1 2601
1 2602
1 2603
1 2604
1 2605
1 2606
1 2607
1 2608
1 2609
1 2610
1 2611
1 2612
1 2613
1 2614
1 2615
1 2616
1 2617
1 2618
1 2619
1 2620
1 2621
1 2622
1 2623
1 2624
1 2625
1 2626
1 2627
1 2628
1 2629
1 2630
1 2631
1 2632
1 2633
1 2634
1 2635
1 2636
1 2637
1 2638
1 2639
1 2640
1 2641
1 2642
1 2643

1 3672
1 3673
1 3674
1 3675
1 3676
1 3677
1 3678
1 3679
1 3680
1 3681
1 3682
1 3683
1 3684
1 3685
1 3686
1 3687
1 3688
1 3689
1 3690
1 3691
1 3692
1 3693
1 3694
1 3695
1 3696
1 3697
1 3698
1 3699
1 3700
1 3701
1 3702
1 3703
1 3704
1 3705
1 3706
1 3707
1 3708
1 3709
1 3710
1 3711
1 3712
1 3713
1 3714
1 3715
1 3716
1 3717
1 3718
1 3719
1 3720
1 3721
1 3722
1 3723
1 3724
1 3725
1 3726
1 3727
1 3728
1 3729
1 3730
1 3731
1 3732
1 3733
1 3734
1 3735
1 3736
1 3737
1 3738
1 3739
1 3740
1 3741
1 3742
1 3743
1 3744
1 3745
1 3746
1 3747
1 3748
1 3749
1 3750
1 3751
1 3752
1 3753
1 3754
1 3755
1 3756
1 3757
1 3758
1 3759
1 3760
1 3761
1 3762
1 3763
1 3764
1 3765
1 3766
1 3767
1 3768
1 3769
1 3770
1 3771
1 3772
1 3773
1 3774
1 3775
1 3776
1 3777
1 3778
1 3779
1 3780
1 3781
1 3782
1 3783
1 3784
1 3785
1 3786
1 3787
1 3788
1 3789
1 3790
1 3791
1 3792
1 3793
1 3794
1 3795
1 3796
1 3797
1 3798
1 3799
1 3800
1 3801
1 3802
1 3803
1 3804
1 3805
1 3806
1 3807
1 3808
1 3809
1 3810
1 3811
1 3812
1 3813
1 3814

1 4844
1 4845
1 4846
1 4847
1 4848
1 4849
1 4850
1 4851
1 4852
1 4853
1 4854
1 4855
1 4856
1 4857
1 4858
1 4859
1 4860
1 4861
1 4862
1 4863
1 4864
1 4865
1 4866
1 4867
1 4868
1 4869
1 4870
1 4871
1 4872
1 4873
1 4874
1 4875
1 4876
1 4877
1 4878
1 4879
1 4880
1 4881
1 4882
1 4883
1 4884
1 4885
1 4886
1 4887
1 4888
1 4889
1 4890
1 4891
1 4892
1 4893
1 4894
1 4895
1 4896
1 4897
1 4898
1 4899
1 4900
1 4901
1 4902
1 4903
1 4904
1 4905
1 4906
1 4907
1 4908
1 4909
1 4910
1 4911
1 4912
1 4913
1 4914
1 4915
1 4916
1 4917
1 4918
1 4919
1 4920
1 4921
1 4922
1 4923
1 4924
1 4925
1 4926
1 4927
1 4928
1 4929
1 4930
1 4931
1 4932
1 4933
1 4934
1 4935
1 4936
1 4937
1 4938
1 4939
1 4940
1 4941
1 4942
1 4943
1 4944
1 4945
1 4946
1 4947
1 4948
1 4949
1 4950
1 4951
1 4952
1 4953
1 4954
1 4955
1 4956
1 4957
1 4958
1 4959
1 4960
1 4961
1 4962
1 4963
1 4964
1 4965
1 4966
1 4967
1 4968
1 4969
1 4970
1 4971
1 4972
1 4973
1 4974
1 4975
1 4976
1 4977
1 4978
1 4979
1 4980
1 4981
1 4982
1 4983
1 4984
1 4985
1 4986

...

3 7413
3 7414
3 7415
3 7416
3 7417
3 7418
3 7419
3 7420
3 7421
3 7422
3 7423
3 7424
3 7425
3 7426
3 7427
3 7428
3 7429
3 7430
3 7431
3 7432
3 7433
3 7434
3 7435
3 7436
3 7437
3 7438
3 7439
3 7440
3 7441
3 7442
3 7443
3 7444
3 7445
3 7446
3 7447
3 7448
3 7449
3 7450
3 7451
3 7452
3 7453
3 7454
3 7455
3 7456
3 7457
3 7458
3 7459
3 7460
3 7461
3 7462
3 7463
3 7464
3 7465
3 7466
3 7467
3 7468
3 7469
3 7470
3 7471
3 7472
3 7473
3 7474
3 7475
3 7476
3 7477
3 7478
3 7479
3 7480
3 7481
3 7482
3 7483
3 7484
3 7485
3 7486
3 7487
3 7488
3 7489
3 7490
3 7491
3 7492
3 7493
3 7494
3 7495
3 7496
3 7497
3 7498
3 7499
3 7500
3 7501
3 7502
3 7503
3 7504
3 7505
3 7506
3 7507
3 7508
3 7509
3 7510
3 7511
3 7512
3 7513
3 7514
3 7515
3 7516
3 7517
3 7518
3 7519
3 7520
3 7521
3 7522
3 7523
3 7524
3 7525
3 7526
3 7527
3 7528
3 7529
3 7530
3 7531
3 7532
3 7533
3 7534
3 7535
3 7536
3 7537
3 7538
3 7539
3 7540
3 7541
3 7542
3 7543
3 7544
3 7545
3 7546
3 7547
3 7548
3 7549
3 7550
3 7551
3 7552
3 7553
3 7554
3 7555

3 8585
3 8586
3 8587
3 8588
3 8589
3 8590
3 8591
3 8592
3 8593
3 8594
3 8595
3 8596
3 8597
3 8598
3 8599
3 8600
3 8601
3 8602
3 8603
3 8604
3 8605
3 8606
3 8607
3 8608
3 8609
3 8610
3 8611
3 8612
3 8613
3 8614
3 8615
3 8616
3 8617
3 8618
3 8619
3 8620
3 8621
3 8622
3 8623
3 8624
3 8625
3 8626
3 8627
3 8628
3 8629
3 8630
3 8631
3 8632
3 8633
3 8634
3 8635
3 8636
3 8637
3 8638
3 8639
3 8640
3 8641
3 8642
3 8643
3 8644
3 8645
3 8646
3 8647
3 8648
3 8649
3 8650
3 8651
3 8652
3 8653
3 8654
3 8655
3 8656
3 8657
3 8658
3 8659
3 8660
3 8661
3 8662
3 8663
3 8664
3 8665
3 8666
3 8667
3 8668
3 8669
3 8670
3 8671
3 8672
3 8673
3 8674
3 8675
3 8676
3 8677
3 8678
3 8679
3 8680
3 8681
3 8682
3 8683
3 8684
3 8685
3 8686
3 8687
3 8688
3 8689
3 8690
3 8691
3 8692
3 8693
3 8694
3 8695
3 8696
3 8697
3 8698
3 8699
3 8700
3 8701
3 8702
3 8703
3 8704
3 8705
3 8706
3 8707
3 8708
3 8709
3 8710
3 8711
3 8712
3 8713
3 8714
3 8715
3 8716
3 8717
3 8718
3 8719
3 8720
3 8721
3 8722
3 8723
3 8724
3 8725
3 8726
3 8727

3 9756
3 9757
3 9758
3 9759
3 9760
3 9761
3 9762
3 9763
3 9764
3 9765
3 9766
3 9767
3 9768
3 9769
3 9770
3 9771
3 9772
3 9773
3 9774
3 9775
3 9776
3 9777
3 9778
3 9779
3 9780
3 9781
3 9782
3 9783
3 9784
3 9785
3 9786
3 9787
3 9788
3 9789
3 9790
3 9791
3 9792
3 9793
3 9794
3 9795
3 9796
3 9797
3 9798
3 9799
3 9800
3 9801
3 9802
3 9803
3 9804
3 9805
3 9806
3 9807
3 9808
3 9809
3 9810
3 9811
3 9812
3 9813
3 9814
3 9815
3 9816
3 9817
3 9818
3 9819
3 9820
3 9821
3 9822
3 9823
3 9824
3 9825
3 9826
3 9827
3 9828
3 9829
3 9830
3 9831
3 9832
3 9833
3 9834
3 9835
3 9836
3 9837
3 9838
3 9839
3 9840
3 9841
3 9842
3 9843
3 9844
3 9845
3 9846
3 9847
3 9848
3 9849
3 9850
3 9851
3 9852
3 9853
3 9854
3 9855
3 9856
3 9857
3 9858
3 9859
3 9860
3 9861
3 9862
3 9863
3 9864
3 9865
3 9866
3 9867
3 9868
3 9869
3 9870
3 9871
3 9872
3 9873
3 9874
3 9875
3 9876
3 9877
3 9878
3 9879
3 9880
3 9881
3 9882
3 9883
3 9884
3 9885
3 9886
3 9887
3 9888
3 9889
3 9890
3 9891
3 9892
3 9893
3 9894
3 9895
3 9896
3 9897
3 9898

4 1086
4 1087
4 1088
4 1089
4 1090
4 1091
4 1092
4 1093
4 1094
4 1095
4 1096
4 1097
4 1098
4 1099
4 1100
4 1101
4 1102
4 1103
4 1104
4 1105
4 1106
4 1107
4 1108
4 1109
4 1110
4 1111
4 1112
4 1113
4 1114
4 1115
4 1116
4 1117
4 1118
4 1119
4 1120
4 1121
4 1122
4 1123
4 1124
4 1125
4 1126
4 1127
4 1128
4 1129
4 1130
4 1131
4 1132
4 1133
4 1134
4 1135
4 1136
4 1137
4 1138
4 1139
4 1140
4 1141
4 1142
4 1143
4 1144
4 1145
4 1146
4 1147
4 1148
4 1149
4 1150
4 1151
4 1152
4 1153
4 1154
4 1155
4 1156
4 1157
4 1158
4 1159
4 1160
4 1161
4 1162
4 1163
4 1164
4 1165
4 1166
4 1167
4 1168
4 1169
4 1170
4 1171
4 1172
4 1173
4 1174
4 1175
4 1176
4 1177
4 1178
4 1179
4 1180
4 1181
4 1182
4 1183
4 1184
4 1185
4 1186
4 1187
4 1188
4 1189
4 1190
4 1191
4 1192
4 1193
4 1194
4 1195
4 1196
4 1197
4 1198
4 1199
4 1200
4 1201
4 1202
4 1203
4 1204
4 1205
4 1206
4 1207
4 1208
4 1209
4 1210
4 1211
4 1212
4 1213
4 1214
4 1215
4 1216
4 1217
4 1218
4 1219
4 1220
4 1221
4 1222
4 1223
4 1224
4 1225
4 1226
4 1227
4 1228

4 2257
4 2258
4 2259
4 2260
4 2261
4 2262
4 2263
4 2264
4 2265
4 2266
4 2267
4 2268
4 2269
4 2270
4 2271
4 2272
4 2273
4 2274
4 2275
4 2276
4 2277
4 2278
4 2279
4 2280
4 2281
4 2282
4 2283
4 2284
4 2285
4 2286
4 2287
4 2288
4 2289
4 2290
4 2291
4 2292
4 2293
4 2294
4 2295
4 2296
4 2297
4 2298
4 2299
4 2300
4 2301
4 2302
4 2303
4 2304
4 2305
4 2306
4 2307
4 2308
4 2309
4 2310
4 2311
4 2312
4 2313
4 2314
4 2315
4 2316
4 2317
4 2318
4 2319
4 2320
4 2321
4 2322
4 2323
4 2324
4 2325
4 2326
4 2327
4 2328
4 2329
4 2330
4 2331
4 2332
4 2333
4 2334
4 2335
4 2336
4 2337
4 2338
4 2339
4 2340
4 2341
4 2342
4 2343
4 2344
4 2345
4 2346
4 2347
4 2348
4 2349
4 2350
4 2351
4 2352
4 2353
4 2354
4 2355
4 2356
4 2357
4 2358
4 2359
4 2360
4 2361
4 2362
4 2363
4 2364
4 2365
4 2366
4 2367
4 2368
4 2369
4 2370
4 2371
4 2372
4 2373
4 2374
4 2375
4 2376
4 2377
4 2378
4 2379
4 2380
4 2381
4 2382
4 2383
4 2384
4 2385
4 2386
4 2387
4 2388
4 2389
4 2390
4 2391
4 2392
4 2393
4 2394
4 2395
4 2396
4 2397
4 2398
4 2399

In [13]:
num_qubits = 4
num_checks = 4

prepcheck_circs_list = []
prepchecks_list = []
for check_id in range(1, num_checks + 1):
    circs = []
    checks = []
    for i in range(total_trials):
        print(check_id, i)
        circ, check = hydrogen_shadow_PCS_checkprep(C_list[i], num_qubits, check_id, True)
        circs.append(circ)
        checks.append(check)
    prepcheck_circs_list.append(circs)
    prepchecks_list.append(checks)

In [14]:
b_lists_filtered = []
check_id = 1
# Submit hardware jobs via Qiskit Runtime;

with Session(backend=backend) as session:
    sampler = Sampler(session=session, options=options)

    # same as the calibration process
    job = sampler.run(circs_list[check_id-1], shots=100, initial_layout=[])
    print(f"Job ID: {job.job_id()}")
    print(f">>> Job Status: {job.status()}")

    result = job.result()

    # Close the session only if all jobs are finished
    # and you don't need to run more in the session.
    session.close()

b_lists_check = []

for i in range(total_trials):
    di = {}
    for key in list(result.quasi_dists[i].binary_probabilities().keys()):
        di.update({key[:num_qubits + check_id]: result.quasi_dists[i].binary_probabilities().get(key)})
    b_lists_check.append(di)


filtered_b_lists = []
for i in range(total_trials):
    bit_list = ['1' if i == '+1' else '0' for i in checks_list[check_id-1][i][0][check_id - 1::-1]]
#     print(bit_list)
    output_dist = rightchecks_postprocess(b_lists_check[i], num_qubits, check_id, pr_list = checks_list[check_id - 1][i][2])
    filted_dist = filter_results_reindex(output_dist, num_qubits, [j for j in range(0, check_id)], bit_list)
    print(total_counts(filted_dist))
    filtered_b_lists.append(filted_dist)
b_lists_filtered.append(filtered_b_lists)


In [15]:
b_lists_check[1]

In [16]:
output_dist = rightchecks_postprocess(b_lists_check[1], num_qubits, check_id, pr_list = checks_list[check_id - 1][1][2])
output_dist

In [17]:
circs_list[0][1].draw()

In [18]:
bit_list = ['1' if i == '+1' else '0' for i in checks_list[check_id-1][1][0][check_id - 1::-1]]
bit_list

In [19]:
filted_dist = filter_results_reindex(output_dist, num_qubits, [j for j in range(0, check_id)], bit_list)
filted_dist

In [20]:
len(circs_list[0])

In [21]:
len(prepcheck_circs_list[0])

In [22]:
circs_list[1][0].draw()

In [23]:
prepcheck_circs_list[1][0].draw()

In [24]:
prepcheck_circs_list[0][5].draw()

In [25]:
prep_b_lists_filtered = []
check_id = 1
# Submit hardware jobs via Qiskit Runtime;

with Session(backend=backend) as session:
    sampler = Sampler(session=session, options=options)

    # same as the calibration process
    job = sampler.run( prepcheck_circs_list[check_id-1], shots=100, initial_layout=[])
    print(f"Job ID: {job.job_id()}")
    print(f">>> Job Status: {job.status()}")

    result = job.result()

    # Close the session only if all jobs are finished
    # and you don't need to run more in the session.
    session.close()

prep_b_lists_check = []

for i in range(total_trials):
    di = {}
    for key in list(result.quasi_dists[i].binary_probabilities().keys()):
        di.update({key[:num_qubits + check_id]: result.quasi_dists[i].binary_probabilities().get(key)})
    prep_b_lists_check.append(di)


prep_filtered_b_lists = []
for i in range(total_trials):
    bit_list = ['1' if i == '+1' else '0' for i in prepchecks_list[check_id-1][i][0][check_id - 1::-1]]
#     print(bit_list)
    prep_output_dist = rightchecks_postprocess(prep_b_lists_check[i], num_qubits, check_id, pr_list = prepchecks_list[check_id - 1][i][2])
    prep_filted_dist = filter_results_reindex(prep_output_dist, num_qubits, [j for j in range(0, check_id)], bit_list)
    print(i, total_counts(prep_filted_dist))
    prep_filtered_b_lists.append(prep_filted_dist)
prep_b_lists_filtered.append(prep_filtered_b_lists)


In [26]:
check_id = 2
# Submit hardware jobs via Qiskit Runtime;

with Session(backend=backend) as session:
    sampler = Sampler(session=session, options=options)

    # same as the calibration process
    job = sampler.run(circs_list[check_id-1], shots=100, initial_layout=[])
    print(f"Job ID: {job.job_id()}")
    print(f">>> Job Status: {job.status()}")

    result = job.result()

    # Close the session only if all jobs are finished
    # and you don't need to run more in the session.
    session.close()

b_lists_check = []

for i in range(total_trials):
    di = {}
    for key in list(result.quasi_dists[i].binary_probabilities().keys()):
        di.update({key[:num_qubits + check_id]: result.quasi_dists[i].binary_probabilities().get(key)})
    b_lists_check.append(di)


filtered_b_lists = []
for i in range(total_trials):
    bit_list = ['1' if i == '+1' else '0' for i in checks_list[check_id-1][i][0][check_id - 1::-1]]
#     print(bit_list)
    output_dist = rightchecks_postprocess(b_lists_check[i], num_qubits, check_id, pr_list = checks_list[check_id - 1][i][2])
    filted_dist = filter_results_reindex(output_dist, num_qubits, [j for j in range(0, check_id)], bit_list)
    print(total_counts(filted_dist))
    filtered_b_lists.append(filted_dist)
b_lists_filtered.append(filtered_b_lists)


In [27]:
prepcheck_circs_list[2-1][0].draw()

In [28]:

check_id = 2
# Submit hardware jobs via Qiskit Runtime;

with Session(backend=backend) as session:
    sampler = Sampler(session=session, options=options)

    # same as the calibration process
    job = sampler.run( prepcheck_circs_list[check_id-1], shots=100, initial_layout=[])
    print(f"Job ID: {job.job_id()}")
    print(f">>> Job Status: {job.status()}")

    result = job.result()

    # Close the session only if all jobs are finished
    # and you don't need to run more in the session.
    session.close()

prep_b_lists_check = []

for i in range(total_trials):
    di = {}
    for key in list(result.quasi_dists[i].binary_probabilities().keys()):
        di.update({key[:num_qubits + check_id]: result.quasi_dists[i].binary_probabilities().get(key)})
    prep_b_lists_check.append(di)


prep_filtered_b_lists = []
for i in range(total_trials):
    bit_list = ['1' if i == '+1' else '0' for i in prepchecks_list[check_id-1][i][0][check_id - 1::-1]]
#     print(bit_list)
    prep_output_dist = rightchecks_postprocess(prep_b_lists_check[i], num_qubits, check_id, pr_list = prepchecks_list[check_id - 1][i][2])
    prep_filted_dist = filter_results_reindex(prep_output_dist, num_qubits, [j for j in range(0, check_id)], bit_list)
    print(i, total_counts(prep_filted_dist))
    prep_filtered_b_lists.append(prep_filted_dist)
prep_b_lists_filtered.append(prep_filtered_b_lists)


In [29]:

check_id = 3
# Submit hardware jobs via Qiskit Runtime;

with Session(backend=backend) as session:
    sampler = Sampler(session=session, options=options)

    # same as the calibration process
    job = sampler.run(circs_list[check_id-1], shots=100, initial_layout=[])
    print(f"Job ID: {job.job_id()}")
    print(f">>> Job Status: {job.status()}")

    result = job.result()

    # Close the session only if all jobs are finished
    # and you don't need to run more in the session.
    session.close()

b_lists_check = []

for i in range(total_trials):
    di = {}
    for key in list(result.quasi_dists[i].binary_probabilities().keys()):
        di.update({key[:num_qubits + check_id]: result.quasi_dists[i].binary_probabilities().get(key)})
    b_lists_check.append(di)


filtered_b_lists = []
for i in range(total_trials):
    bit_list = ['1' if i == '+1' else '0' for i in checks_list[check_id-1][i][0][check_id - 1::-1]]
#     print(bit_list)
    output_dist = rightchecks_postprocess(b_lists_check[i], num_qubits, check_id, pr_list = checks_list[check_id - 1][i][2])
    filted_dist = filter_results_reindex(output_dist, num_qubits, [j for j in range(0, check_id)], bit_list)
    print(total_counts(filted_dist))
    filtered_b_lists.append(filted_dist)
b_lists_filtered.append(filtered_b_lists)


In [30]:

check_id = 3
# Submit hardware jobs via Qiskit Runtime;

with Session(backend=backend) as session:
    sampler = Sampler(session=session, options=options)

    # same as the calibration process
    job = sampler.run( prepcheck_circs_list[check_id-1], shots=100, initial_layout=[])
    print(f"Job ID: {job.job_id()}")
    print(f">>> Job Status: {job.status()}")

    result = job.result()

    # Close the session only if all jobs are finished
    # and you don't need to run more in the session.
    session.close()

prep_b_lists_check = []

for i in range(total_trials):
    di = {}
    for key in list(result.quasi_dists[i].binary_probabilities().keys()):
        di.update({key[:num_qubits + check_id]: result.quasi_dists[i].binary_probabilities().get(key)})
    prep_b_lists_check.append(di)


prep_filtered_b_lists = []
for i in range(total_trials):
    bit_list = ['1' if i == '+1' else '0' for i in prepchecks_list[check_id-1][i][0][check_id - 1::-1]]
#     print(bit_list)
    prep_output_dist = rightchecks_postprocess(prep_b_lists_check[i], num_qubits, check_id, pr_list = prepchecks_list[check_id - 1][i][2])
    prep_filted_dist = filter_results_reindex(prep_output_dist, num_qubits, [j for j in range(0, check_id)], bit_list)
    print(i, total_counts(prep_filted_dist))
    prep_filtered_b_lists.append(prep_filted_dist)
prep_b_lists_filtered.append(prep_filtered_b_lists)


In [31]:

check_id = 4
# Submit hardware jobs via Qiskit Runtime;

with Session(backend=backend) as session:
    sampler = Sampler(session=session, options=options)

    # same as the calibration process
    job = sampler.run(circs_list[check_id-1], shots=100, initial_layout=[])
    print(f"Job ID: {job.job_id()}")
    print(f">>> Job Status: {job.status()}")

    result = job.result()

    # Close the session only if all jobs are finished
    # and you don't need to run more in the session.
    session.close()

b_lists_check = []

for i in range(total_trials):
    di = {}
    for key in list(result.quasi_dists[i].binary_probabilities().keys()):
        di.update({key[:num_qubits + check_id]: result.quasi_dists[i].binary_probabilities().get(key)})
    b_lists_check.append(di)


filtered_b_lists = []
for i in range(total_trials):
    bit_list = ['1' if i == '+1' else '0' for i in checks_list[check_id-1][i][0][check_id - 1::-1]]
#     print(bit_list)
    output_dist = rightchecks_postprocess(b_lists_check[i], num_qubits, check_id, pr_list = checks_list[check_id - 1][i][2])
    filted_dist = filter_results_reindex(output_dist, num_qubits, [j for j in range(0, check_id)], bit_list)
    print(total_counts(filted_dist))
    filtered_b_lists.append(filted_dist)
b_lists_filtered.append(filtered_b_lists)


In [32]:
len(b_lists_check)

In [33]:

check_id = 4
# Submit hardware jobs via Qiskit Runtime;

with Session(backend=backend) as session:
    sampler = Sampler(session=session, options=options)

    # same as the calibration process
    job = sampler.run( prepcheck_circs_list[check_id-1], shots=100, initial_layout=[])
    print(f"Job ID: {job.job_id()}")
    print(f">>> Job Status: {job.status()}")

    result = job.result()

    # Close the session only if all jobs are finished
    # and you don't need to run more in the session.
    session.close()

prep_b_lists_check = []

for i in range(total_trials):
    di = {}
    for key in list(result.quasi_dists[i].binary_probabilities().keys()):
        di.update({key[:num_qubits + check_id]: result.quasi_dists[i].binary_probabilities().get(key)})
    prep_b_lists_check.append(di)


prep_filtered_b_lists = []
for i in range(total_trials):
    bit_list = ['1' if i == '+1' else '0' for i in prepchecks_list[check_id-1][i][0][check_id - 1::-1]]
#     print(bit_list)
    prep_output_dist = rightchecks_postprocess(prep_b_lists_check[i], num_qubits, check_id, pr_list = prepchecks_list[check_id - 1][i][2])
    prep_filted_dist = filter_results_reindex(prep_output_dist, num_qubits, [j for j in range(0, check_id)], bit_list)
    print(i, total_counts(prep_filted_dist))
    prep_filtered_b_lists.append(prep_filted_dist)
prep_b_lists_filtered.append(prep_filtered_b_lists)


In [34]:
# Submit hardware jobs via Qiskit Runtime;

with Session(backend=backend) as session:
    sampler = Sampler(session=session, options=options)
    
    # same as the calibration process
    job = sampler.run(orign_circs, shots=100, initial_layout=[])
    print(f"Job ID: {job.job_id()}")
    print(f">>> Job Status: {job.status()}")
    
    result = job.result()
    
    # Close the session only if all jobs are finished
    # and you don't need to run more in the session.
    session.close()

In [35]:
b_lists = []

for i in range(total_trials):
    di = {}
    for key in list(result.quasi_dists[i].binary_probabilities().keys()):
        di.update({key[:num_qubits + num_checks]: result.quasi_dists[i].binary_probabilities().get(key)})
    b_lists.append(di)

Noiseless Experiments on qiskitruntime

In [36]:
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Sampler, Options

options = Options(optimization_level=2, resilience_level=1)
# backend = service.get_backend("ibmq_qasm_simulator")

In [37]:
with Session(backend=backend) as session:
    sampler = Sampler(session=session, options=options)

    job = sampler.run(orign_circs, shots=100)
    print(f"Job ID: {job.job_id()}")
    print(f">>> Job Status: {job.status()}")
    
    result = job.result()
    
    # Close the session only if all jobs are finished
    # and you don't need to run more in the session.
    session.close()

In [38]:
b_lists_noiseless = []

for i in range(total_trials):
    di = {}
    for key in list(result.quasi_dists[i].binary_probabilities().keys()):
        di.update({key[:num_qubits]: result.quasi_dists[i].binary_probabilities().get(key)})
    b_lists_noiseless.append(di)

In [39]:
def compute_expectation(b_lists, b_lists_checks, b_lists_noiseless, C_list, operator_list, num_qubits, f_tilde):
    """
    Reconstruct a state approximation as an average over all snapshots in the shadow.
    Args:
        shadow (tuple): A shadow tuple obtained from `calculate_classical_shadow`.
        operator (np.ndarray):
        num_qubits
    Returns:
        Numpy array with the reconstructed quantum state.
    """
    num_snapshots = len(b_lists)
    
    # Averaging over snapshot states.
    expectation_list = np.zeros(len(operator_list))
    expectation_list_r = np.zeros(len(operator_list))
    expectation_list_checks = [np.zeros(len(operator_list)) for i in range(len(b_lists_checks))]
    expectation_list_noiseless = np.zeros(len(operator_list))
    
    for i in range(num_snapshots):
        noisy, robust = expectation_snapshot(b_lists[i], C_list[i], operator_list, num_qubits, f_tilde)
        expectation_list += noisy
        expectation_list_r += robust
        
        for j in range(len(b_lists_checks)):
            check = expectation_snapshot_noiseless(b_lists_checks[j][i], C_list[i], operator_list, num_qubits)
            expectation_list_checks[j] += check 
            
        
        noiseless = expectation_snapshot_noiseless(b_lists_noiseless[i], C_list[i], operator_list, num_qubits)
        expectation_list_noiseless += noiseless
        
    expectation_list /= num_snapshots
    expectation_list_r /= num_snapshots
    for j in range(len(b_lists_checks)):
        expectation_list_checks[j] /= num_snapshots
    expectation_list_noiseless /= num_snapshots
    
    return expectation_list, expectation_list_r, expectation_list_checks, expectation_list_noiseless


def expectation_snapshot(b_dict, clifford, operator_list, num_qubits, f_tilde):
    """
    Helper function for `shadow_state_reconstruction` that reconstructs the overlap estimate from
    a single snapshot in a shadow. Implements Eq. (S23) from https://arxiv.org/pdf/2106.16235.pdf
    Args:
        b_dict (dict): The list of classical outcomes for the snapshot.
        clifford: Indices for the applied Pauli measurement.
        operator:
        num_qubits:
    Returns:
        Numpy array with the reconstructed snapshot.
    """
    f = 1/(2**num_qubits+1)
    # reconstructing the snapshot state from random Clifford measurements
    U = clifford.to_matrix()
    I = np.eye(2**num_qubits)
    
    # applying Eq. (S32), note that this expression is built upon random Clifford, so that inverting
    # the quantum channel follows Eq. (S29).
    snapshot_list = np.zeros(len(operator_list))
    snapshot_list_r = np.zeros(len(operator_list))
    denom = 0
    for b_state in list(b_dict.keys()):
        matrix_part = U.conj().T @ state_reconstruction(b_state) @ U
        
        interm = 1 / f * matrix_part
        interm -= (1 / f - 1)/2**num_qubits * I
        
        interm_r = 1 / f_tilde * matrix_part
        interm_r -= (1 / f_tilde - 1) / 2**num_qubits * I
        
        for index, operator in enumerate(operator_list):
            operator_matrix = operator.to_matrix()
            snapshot_list[index] += np.real(np.trace(operator_matrix @ interm) * b_dict.get(b_state))
            snapshot_list_r[index] += np.real(np.trace(operator_matrix @ interm_r) * b_dict.get(b_state))
            
        denom += b_dict.get(b_state)
    
    return snapshot_list / denom, snapshot_list_r / denom


def expectation_snapshot_noiseless(b_dict, clifford, operator_list, num_qubits):
    """
    Helper function for `shadow_state_reconstruction` that reconstructs the overlap estimate from
    a single snapshot in a shadow. Implements Eq. (S23) from https://arxiv.org/pdf/2106.16235.pdf
    Args:
        b_dict (dict): The list of classical outcomes for the snapshot.
        clifford: Indices for the applied Pauli measurement.
        operator:
        num_qubits:
    Returns:
        Numpy array with the reconstructed snapshot.
    """
    f = 1/(2**num_qubits+1)
    # reconstructing the snapshot state from random Clifford measurements
    U = clifford.to_matrix()
    I = np.eye(2**num_qubits)
    
    # applying Eq. (S32), note that this expression is built upon random Clifford, so that inverting
    # the quantum channel follows Eq. (S29).
    snapshot_list = np.zeros(len(operator_list))
    denom = 0
    for b_state in list(b_dict.keys()):
        interm = 1/f * U.conj().T @ state_reconstruction(b_state) @ U
        interm -= (1/f - 1)/2**num_qubits * I
        
        for index, operator in enumerate(operator_list):
            operator_matrix = operator.to_matrix()
            snapshot_list[index] += np.real(np.trace(operator_matrix @ interm) * b_dict.get(b_state))
            
        denom += b_dict.get(b_state)
    
    return snapshot_list / denom

In [40]:
# run the classical shadows postprocessing to get expectation values;

Paulis = ['XXXX', 'YYYY', 'XYXY', 'YXYX', 'YYXX', 'XXYY', 'ZZZZ', 'ZZII', 'IIZZ',
         'XZXZ', 'ZXZX', 'ZZXX', 'XXZZ', 'IIXX', 'XXII', 'XIIX']

operator_list = []
for pauli in Paulis:
    operator_list.append(Pauli(pauli))

psi = Statevector(hydrogen_trial_circuit(num_qubits))
ref_list = []
for operator in operator_list:
    expect = np.array(psi).T.conj() @ operator.to_matrix() @ np.array(psi)
    ref_list.append(expect)

In [41]:
# b_sublists_dict.keys()

In [42]:
num_of_runs = 20
shadow_range = [100, 400, 1000, 4000, 10000]#, 40000]
num_of_checks = 4

# Simplify initialization of expectation arrays
expectations = {
    name: np.zeros((len(shadow_range), len(Paulis), num_of_runs))
    for name in ["shadow", "shadow_r", "noiseless"]
    + [f"check{k+1}" for k in range(num_of_checks)]
    + [f"prepcheck{k+1}" for k in range(num_of_checks)]
}

for j, num_snapshots in enumerate(shadow_range):
    indices = random.sample(range(total_trials), num_snapshots)
    print(f'num total snapshots = {num_snapshots}')

    # Partition indices into 'num_of_runs' equally sized chunks
    partitions = np.array_split(indices, num_of_runs)
    # print('paritions', partitions)
    # print(len(partitions))

    for i, run_indices in enumerate(partitions):
        print('i = ', i)
        # Create sublists using list comprehensions
        C_run_sublist = [C_list[idx] for idx in run_indices]
        b_run_sublists_dict = {
            "base": [b_lists[idx] for idx in run_indices],
            "noiseless": [b_lists_noiseless[idx] for idx in run_indices],
            "checks": [[b_lists_filtered[k][idx] for idx in run_indices] for k in range(num_of_checks)],
            "prepchecks": [[prep_b_lists_filtered[k][idx] for idx in run_indices] for k in range(num_of_checks)],
        }

        # Compute expectations for each type of check
        expectation_list, expectation_list_r, expectation_list_checks, expectation_list_noiseless = compute_expectation(
            b_run_sublists_dict['base'], b_run_sublists_dict['checks'], b_run_sublists_dict['noiseless'], C_run_sublist, operator_list, 4, f_tilde
        )
        expectation_list, expectation_list_r, expectation_list_prepchecks, expectation_list_noiseless = compute_expectation(
            b_run_sublists_dict['base'], b_run_sublists_dict['prepchecks'], b_run_sublists_dict['noiseless'], C_run_sublist, operator_list, 4, f_tilde
        )

        # Store the results
        expectations['shadow'][j, :, i] = np.real(expectation_list)
        expectations['shadow_r'][j, :, i] = np.real(expectation_list_r)
        expectations['noiseless'][j, :, i] = np.real(expectation_list_noiseless)
        for k in range(num_of_checks):
            expectations[f"check{k+1}"][j, :, i] = np.real(expectation_list_checks[k])
        for k in range(num_of_checks):
            expectations[f"prepcheck{k+1}"][j, :, i] = np.real(expectation_list_prepchecks[k])

In [43]:
# Save the expectations dictionary to a .npz file
np.savez('expectations_fakeCairo.npz', **expectations)

In [44]:
# num_of_runs = 20
# shadow_range = [10, 50, 100, 500]#, 4000]
# num_of_checks = 4
# # Simplify initialization of expectation arrays
# expectations = {
#     name: np.zeros((len(shadow_range), len(Paulis), num_of_runs))
#     for name in ["shadow", "shadow_r", "noiseless"]
#     + [f"check{k+1}" for k in range(num_of_checks)]
#     + [f"prepcheck{k+1}" for k in range(num_of_checks)]
# }

# for i in range(num_of_runs):
#     for j, num_snapshots in enumerate(shadow_range):
#         indices = random.sample(range(total_trials), num_snapshots)
#         print(f'run # {i}, num snapshots = {num_snapshots}')
        
#         # Create sublists using list comprehensions
#         C_sublist = [C_list[idx] for idx in indices]
#         b_sublists_dict = {
#             "base": [b_lists[idx] for idx in indices],
#             "noiseless": [b_lists_noiseless[idx] for idx in indices],
#             "checks": [[b_lists_filtered[k][idx] for idx in indices] for k in range(num_of_checks)],
#             "prepchecks": [[prep_b_lists_filtered[k][idx] for idx in indices] for k in range(num_of_checks)],
#         }
        
#         # Compute expectations for each type of check
        
#         expectation_list, expectation_list_r, expectation_list_checks, expectation_list_noiseless = compute_expectation(
#             b_sublists_dict['base'], b_sublists_dict['checks'],  b_sublists_dict['noiseless'], C_sublist, operator_list, 4, f_tilde
#         )
#         expectation_list, expectation_list_r, expectation_list_prepchecks, expectation_list_noiseless = compute_expectation(
#             b_sublists_dict['base'], b_sublists_dict['prepchecks'],  b_sublists_dict['noiseless'], C_sublist, operator_list, 4, f_tilde
#         )
    
#         expectations['shadow'][j, :, i] = np.real(expectation_list)
#         expectations['shadow_r'][j, :, i] = np.real(expectation_list_r)  
#         expectations['noiseless'][j, :, i] = np.real(expectation_list_noiseless)  
#         for k in range(num_of_checks):
#             expectations[f"check{k+1}"][j, :, i] = np.real(expectation_list_checks[k])
#         for k in range(num_of_checks):
#             expectations[f"prepcheck{k+1}"][j, :, i] = np.real(expectation_list_prepchecks[k])
        


In [45]:
errors = {
    name: np.zeros(len(shadow_range))
    for name in ["shadow", "shadow_r", "noiseless"]
    + [f"check{k+1}" for k in range(num_of_checks)]
    + [f"prepcheck{k+1}" for k in range(num_of_checks)]
}
for i in range(len(shadow_range)):
    errors['shadow'][i] = np.mean([np.abs(np.median(expectations["shadow"][i], axis=1)[j] - ref_list[j]) for j in range(len(ref_list))])
    errors['shadow_r'][i] = np.mean([np.abs(np.median(expectations["shadow_r"][i], axis=1)[j] - ref_list[j]) for j in range(len(ref_list))])  
    errors['noiseless'][i] = np.mean([np.abs(np.median(expectations["noiseless"][i], axis=1)[j] - ref_list[j]) for j in range(len(ref_list))])
    for k in range(num_of_checks):
        errors[f"check{k+1}"][i] = np.mean([np.abs(np.median(expectations[f"check{k+1}"][i], axis=1)[j] - ref_list[j]) for j in range(len(ref_list))])
    for k in range(num_of_checks):
        errors[f"prepcheck{k+1}"][i] = np.mean([np.abs(np.median(expectations[f"prepcheck{k+1}"][i], axis=1)[j] - ref_list[j]) for j in range(len(ref_list))])

In [46]:
# error = np.zeros(len(shadow_range))
# error_r = np.zeros(len(shadow_range))
# error_check1 = np.zeros(len(shadow_range))
# error_check2 = np.zeros(len(shadow_range))
# error_check3 = np.zeros(len(shadow_range))
# error_check4 = np.zeros(len(shadow_range))
# error_prepcheck1 = np.zeros(len(shadow_range))
# error_prepcheck2 = np.zeros(len(shadow_range))
# error_prepcheck3 = np.zeros(len(shadow_range))
# error_prepcheck4 = np.zeros(len(shadow_range))
# error_noiseless = np.zeros(len(shadow_range))

# for i in range(len(shadow_range)):
#     error[i] = np.mean([np.abs(np.median(expectation_shadow[i], axis=1)[j] - ref_list[j]) for j in range(len(ref_list))])
#     error_r[i] = np.mean([np.abs(np.median(expectation_shadow_r[i], axis=1)[j] - ref_list[j]) for j in range(len(ref_list))])
#     error_check1[i] = np.mean([np.abs(np.median(expectation_shadow_check1[i], axis=1)[j] - ref_list[j]) for j in range(len(ref_list))])
#     error_check2[i] = np.mean([np.abs(np.median(expectation_shadow_check2[i], axis=1)[j] - ref_list[j]) for j in range(len(ref_list))])
#     error_check3[i] = np.mean([np.abs(np.median(expectation_shadow_check3[i], axis=1)[j] - ref_list[j]) for j in range(len(ref_list))])
#     error_check4[i] = np.mean([np.abs(np.median(expectation_shadow_check4[i], axis=1)[j] - ref_list[j]) for j in range(len(ref_list))])
#     error_prepcheck1[i] = np.mean([np.abs(np.median(expectation_shadow_prepcheck1[i], axis=1)[j] - ref_list[j]) for j in range(len(ref_list))])
#     error_prepcheck2[i] = np.mean([np.abs(np.median(expectation_shadow_prepcheck2[i], axis=1)[j] - ref_list[j]) for j in range(len(ref_list))])
#     error_prepcheck3[i] = np.mean([np.abs(np.median(expectation_shadow_prepcheck3[i], axis=1)[j] - ref_list[j]) for j in range(len(ref_list))])
#     error_prepcheck4[i] = np.mean([np.abs(np.median(expectation_shadow_prepcheck4[i], axis=1)[j] - ref_list[j]) for j in range(len(ref_list))])

#     error_noiseless[i] = np.mean([np.abs(np.median(expectation_shadow_noiseless[i], axis=1)[j] - ref_list[j]) for j in range(len(ref_list))])

In [47]:
# error_check1

In [48]:
plt.figure(figsize=(5, 4), dpi=100)
plt.plot(shadow_range, errors['shadow'], '--o', ms=8, color='tab:orange', label='noisy')
plt.plot(shadow_range, errors['shadow_r'], '--^', ms=8, color='tab:green', label='robust')
plt.plot(shadow_range, errors['noiseless'], '--x', ms=8, color='tab:blue', label='noiseless')
plt.plot(shadow_range, errors['check1'], '--o', ms=8, color='tab:red', label='check1')
plt.plot(shadow_range, errors['check2'], '--o', ms=8, color='tab:purple', label='check2')
plt.plot(shadow_range, errors['check3'], '--o', ms=8, color='tab:olive', label='check3')
plt.plot(shadow_range, errors['check4'], '--o', ms=8, color='tab:pink', label='check4')
plt.legend(fontsize=14, loc='best')
plt.xlabel('Shadow size', fontsize=14)
plt.ylabel('Error', fontsize=14)
plt.xscale('log')
plt.yscale('log')
plt.tick_params(labelsize=14)
plt.tight_layout()
plt.savefig('4-qubit_randomclifford_depolar_p2=0.02.png', dpi=100, bbox_inches="tight")
plt.show()

In [49]:
plt.figure(figsize=(5, 4), dpi=100)
plt.plot(shadow_range, errors['shadow'], '--o', ms=8, color='tab:orange', label='noisy')
plt.plot(shadow_range, errors['shadow_r'], '--^', ms=8, color='tab:green', label='robust')
plt.plot(shadow_range, errors['noiseless'], '--x', ms=8, color='tab:blue', label='noiseless')
plt.plot(shadow_range, errors['prepcheck1'], '--o', ms=8, color='tab:red', label='prepcheck1')
plt.plot(shadow_range, errors['prepcheck2'], '--o', ms=8, color='tab:purple', label='prepcheck2')
plt.plot(shadow_range, errors['prepcheck3'], '--o', ms=8, color='tab:olive', label='prepcheck3')
plt.plot(shadow_range, errors['prepcheck4'], '--o', ms=8, color='tab:pink', label='prepcheck4')
plt.legend(fontsize=14, loc='best')
plt.xlabel('Shadow size', fontsize=14)
plt.ylabel('Error', fontsize=14)
plt.xscale('log')
plt.yscale('log')
plt.tick_params(labelsize=14)
plt.tight_layout()
plt.show()

In [50]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.polynomial.polynomial import Polynomial

# medians = [np.median(check, axis=2) for check in [expectations[f"prepcheck{k+1}"] for k in range(num_checks)]]
# # medians = [np.median(check, axis=2) for check in [expectations[f"check{k+1}"] for k in range(num_checks)]]

# shadow_size_index = -1  # largest shadow size
# pauli_index = 6  # Example observable index
# expectation_values = [median[shadow_size_index, pauli_index] for median in medians]

# # Fit a Straight Line
# check_numbers = [1, 2, 3]  # Numeric x-values for fitting
# polynomial = Polynomial.fit(check_numbers, expectation_values, 1)

# extrapolated_check = 4
# extrapolated_value = polynomial(extrapolated_check)

# Plotting
# plt.figure(figsize=(10, 6))
# plt.scatter(check_numbers, expectation_values, color='blue', label='Measured Data')  # Only scatter plot for original data
# plt.plot(np.linspace(1, 4, 4), polynomial(np.linspace(1, 4, 4)), color='red', label='Fitted Line')  # Keep the fitted line
# plt.scatter([extrapolated_check], [extrapolated_value], color='green', label='Extrapolated for 4th Layer')

# plt.xlabel('Number of Check Layers')
# # plt.ylabel(f'Median Expectation Value for {shadow_range[shadow_size_index]} snapshots')
# plt.ylabel('Expectation Value')
# plt.title(f'Extrapolation of Expectation Value for Observable {Paulis[pauli_index]}')
# plt.legend()
# plt.grid(True)
# plt.xticks(np.arange(1, 5))  # Set x-axis ticks in increments of 1
# plt.savefig('extrapolation_ex.png')
# plt.show()

Calculate Extrapolated Checks

In [51]:
check_numbers = [1, 2, 3]  # Original check layers
extrapolation_layers = range(4, 5)  # Layers we extrapolate to
medians = [np.median(check, axis=2) for check in [expectations[f"prepcheck{k+1}"] for k in range(len(check_numbers))]]

# Initialize a three-dimensional array to store extrapolated values
# Dimensions: [extrapolated layers, shadow size, Paulis]
expectation_check_limit = np.zeros((len(extrapolation_layers), len(shadow_range), len(Paulis)))

for layer_index, layer in enumerate(extrapolation_layers):
    for shadow_size_index in range(len(medians[0])):
        for pauli_index in range(medians[0].shape[1]):
            expectation_values = [median[shadow_size_index, pauli_index] for median in medians]
            polynomial = Polynomial.fit(check_numbers, expectation_values, 1)
            # Extrapolate the value for the current layer
            extrapolated_value = polynomial(layer)
            expectation_check_limit[layer_index, shadow_size_index, pauli_index] = extrapolated_value

In [52]:
# # Example Plotting for a specific Pauli index across all shadow sizes
# pauli_index = 1  
# shadow_size_index = -1 # largest shadow size

# plt.figure(figsize=(10, 6))
# # Plotting the extrapolated values for each extrapolated layer for the specific observable and shadow size
# for layer_index, layer in enumerate(extrapolation_layers):
#     plt.plot(shadow_range, expectation_check_limit[layer_index, :, pauli_index], marker='o', linestyle='-', label=f'Extrapolated for {layer}th Layer')

# plt.xlabel('Shadow Size')
# plt.ylabel(f'Extrapolated Median Expectation Value')
# plt.title(f'Extrapolated Values Across Shadow Sizes for Observable Index {pauli_index}')
# plt.legend()
# plt.grid(True)
# plt.show()

Compute mean error across observables for each extrapolated check

In [53]:
error_check_limit = np.zeros((len(extrapolation_layers), len(shadow_range)))

for layer_index, layer in enumerate(extrapolation_layers):
    for shadow_size_index in range(len(shadow_range)):
        # Calculate the mean error for this layer and shadow size across all Pauli indices
        error_check_limit[layer_index, shadow_size_index] = np.mean(
            [np.abs(expectation_check_limit[layer_index, shadow_size_index, pauli_index] - ref_list[pauli_index]) for pauli_index in range(len(Paulis))]
        )

In [54]:
plt.figure(figsize=(5, 4), dpi=100)
plt.plot(shadow_range, errors['shadow'], '--o', ms=8, color='tab:orange', label='noisy')
plt.plot(shadow_range, errors['prepcheck1'], '--o', ms=8, color='tab:red', label='prepcheck1')
plt.plot(shadow_range, errors['prepcheck2'], '--o', ms=8, color='tab:purple', label='prepcheck2')
plt.plot(shadow_range, errors['prepcheck3'], '--o', ms=8, color='tab:olive', label='prepcheck3')
plt.plot(shadow_range, errors['prepcheck4'], '--o', ms=8, color='tab:pink', label='prepcheck4')
plt.plot(shadow_range, errors['shadow_r'], '--^', ms=8, color='tab:green', label='robust')
plt.plot(shadow_range, errors['noiseless'], '--x', ms=8, color='tab:blue', label='noiseless')

# Plotting each layer of extrapolated checks
colors = ['tab:brown', 'tab:gray', 'tab:cyan', 'tab:pink', 'tab:purple']  # Example colors for different layers
for layer_index, layer in enumerate(extrapolation_layers):
    plt.plot(shadow_range, error_check_limit[layer_index, :], '--o', ms=8, color=colors[layer_index % len(colors)], label=f'prepcheck {layer} (extrap)')

# plt.legend(fontsize=14, loc='best')
# plt.xlabel('Shadow size', fontsize=14)
# plt.ylabel('Error', fontsize=14)
# plt.xscale('log')
# plt.yscale('log')
# plt.tick_params(labelsize=14)
# plt.tight_layout()

# plt.savefig('non_ideal_checks.png', dpi=100)
# plt.show()

# Adjust the legend to be outside without altering the figure size
plt.legend(fontsize=10, loc='upper left', bbox_to_anchor=(1.05, 1))
plt.xlabel('Shadow size', fontsize=14)
plt.ylabel('Error', fontsize=14)
plt.xscale('log')
plt.yscale('log')
plt.tick_params(labelsize=14)

# Note: The figure's layout isn't altered with plt.tight_layout() in this case
# Saving the figure with bbox_inches='tight' includes the external legend
plt.savefig('4-qubit_stateprep_depolar_p2=0.02.png', dpi=100, bbox_inches="tight")
plt.show()


In [55]:
plt.figure(figsize=(5, 4), dpi=100)
plt.plot(shadow_range, errors['shadow'], '--o', ms=8, color='tab:orange', label='noisy')
plt.plot(shadow_range, errors['shadow_r'], '--^', ms=8, color='tab:green', label='robust')
plt.plot(shadow_range, errors['noiseless'], '--x', ms=8, color='tab:blue', label='noiseless')
plt.plot(shadow_range, errors['check1'], '--o', ms=8, color='tab:red', label='check1')
plt.plot(shadow_range, errors['check2'], '--o', ms=8, color='tab:purple', label='check2')
plt.plot(shadow_range, errors['check3'], '--o', ms=8, color='tab:olive', label='check3')
plt.plot(shadow_range, errors['check4'], '--o', ms=8, color='tab:pink', label='check4')

# Plotting each layer of extrapolated checks
colors = ['tab:brown', 'tab:gray', 'tab:cyan', 'tab:pink', 'tab:purple']  # Example colors for different layers
for layer_index, layer in enumerate(extrapolation_layers):
    plt.plot(shadow_range, error_check_limit[layer_index, :], '--o', ms=8, color=colors[layer_index % len(colors)], label=f'check {layer} (extrap)')

# Adjust the legend to be outside without altering the figure size
plt.legend(fontsize=10, loc='upper left', bbox_to_anchor=(1.05, 1))
plt.xlabel('Shadow size', fontsize=14)
plt.ylabel('Error', fontsize=14)
plt.xscale('log')
plt.yscale('log')
plt.tick_params(labelsize=14)

plt.savefig('4-qubit_randomclifford_depolar_p2=0.02.png', dpi=100, bbox_inches="tight")
plt.show()